In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
wildfire_dataset_path = '/content/drive/MyDrive/Forest Fire Project/Datasets/Wildfire Detection Image Data'

In [3]:
pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 95.1 MB/s eta 0:00:00


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import time
import onnx

In [5]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# Define the transformations for training and validation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for the model
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Load the dataset
train_data = datasets.ImageFolder(root=f'{wildfire_dataset_path}/train', transform=transform)
val_data = datasets.ImageFolder(root=f'{wildfire_dataset_path}/val', transform=transform)
test_data = datasets.ImageFolder(root=f'{wildfire_dataset_path}/test', transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [6]:
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 2)  # 2 classes: fire, nofire
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)  # Flatten the tensor
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [7]:
# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model initialization
model = CustomCNN().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function for training
def train_model(model, train_loader, val_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_acc = evaluate_model(model, val_loader)
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {running_loss/len(train_loader):.4f}, Val Acc: {val_acc:.2f}%")

    print("Training complete.")

# Function to evaluate model on validation set
def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

# Train the model
train_model(model, train_loader, val_loader, epochs=10)


Epoch [1/10], Train Loss: 0.7008, Val Acc: 95.00%
Epoch [2/10], Train Loss: 0.1614, Val Acc: 96.25%
Epoch [3/10], Train Loss: 0.1038, Val Acc: 97.50%
Epoch [4/10], Train Loss: 0.0805, Val Acc: 97.50%
Epoch [5/10], Train Loss: 0.0650, Val Acc: 97.50%
Epoch [6/10], Train Loss: 0.0721, Val Acc: 95.00%
Epoch [7/10], Train Loss: 0.0647, Val Acc: 96.25%
Epoch [8/10], Train Loss: 0.0332, Val Acc: 97.50%
Epoch [9/10], Train Loss: 0.0312, Val Acc: 97.50%
Epoch [10/10], Train Loss: 0.0405, Val Acc: 97.50%
Training complete.


In [8]:
# Evaluate the model on the test set
test_acc = evaluate_model(model, test_loader)
print(f"Test Accuracy: {test_acc:.2f}%")

Test Accuracy: 95.67%


In [ ]:
!pip install fvcore


In [11]:
# Check the parameters
from torchsummary import summary
summary(model, (3, 224, 224))  # Assuming the input size is 224x224

# For FLOPs and inference time, you can use the same approach as previously shown
import torch
from fvcore.nn import FlopCountAnalysis
import time

# Get FLOPs
flops = FlopCountAnalysis(model, torch.randn(1, 3, 224, 224).to(device))
print("FLOPs:", flops.total())

# Test Inference time
start_time = time.time()
model(torch.randn(1, 3, 224, 224).to(device))  # Run inference on a random image
inference_time = time.time() - start_time
print(f"Average inference time per image: {inference_time:.4f} seconds")


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
              ReLU-2         [-1, 32, 224, 224]               0
         MaxPool2d-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 64, 112, 112]          18,496
              ReLU-5         [-1, 64, 112, 112]               0
         MaxPool2d-6           [-1, 64, 56, 56]               0
            Conv2d-7          [-1, 128, 56, 56]          73,856
              ReLU-8          [-1, 128, 56, 56]               0
         MaxPool2d-9          [-1, 128, 28, 28]               0
           Linear-10                  [-1, 512]      51,380,736
             ReLU-11                  [-1, 512]               0
           Linear-12                    [-1, 2]           1,026
Total params: 51,475,010
Trainable params: 51,475,010
Non-trainable params: 0
-------------------------

FLOPs: 557155328
Average inference time per image: 0.0033 seconds


## ResNet 18

In [12]:
from torchvision.models import resnet18

# Load pretrained ResNet18
resnet_model = resnet18(pretrained=True)

# Modify the final layer for binary classification
num_ftrs = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_ftrs, 2)

# Move to device
resnet_model = resnet_model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 122MB/s]


In [13]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)

In [14]:
num_epochs = 10

for epoch in range(num_epochs):
    resnet_model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = resnet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    resnet_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = resnet_model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_loader):.4f}, Val Acc: {val_acc:.2f}%')

print("Training complete.")


Epoch [1/10], Train Loss: 0.1533, Val Acc: 92.50%
Epoch [2/10], Train Loss: 0.0619, Val Acc: 97.50%
Epoch [3/10], Train Loss: 0.0510, Val Acc: 97.50%
Epoch [4/10], Train Loss: 0.0287, Val Acc: 97.50%
Epoch [5/10], Train Loss: 0.0375, Val Acc: 93.75%
Epoch [6/10], Train Loss: 0.0351, Val Acc: 100.00%
Epoch [7/10], Train Loss: 0.0135, Val Acc: 98.75%
Epoch [8/10], Train Loss: 0.0668, Val Acc: 97.50%
Epoch [9/10], Train Loss: 0.0338, Val Acc: 100.00%
Epoch [10/10], Train Loss: 0.0231, Val Acc: 98.75%
Training complete.


In [15]:
from torchsummary import summary
summary(resnet_model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [16]:
!pip install fvcore  # if not already installed

from fvcore.nn import FlopCountAnalysis, parameter_count_table
import time

# FLOPs calculation
dummy_input = torch.randn(1, 3, 224, 224).to(device)
flops = FlopCountAnalysis(resnet_model, dummy_input)
print("FLOPs:", flops.total())

# Inference time per image
start_time = time.time()
with torch.no_grad():
    for _ in range(100):
        _ = resnet_model(dummy_input)
end_time = time.time()

avg_inference_time = (end_time - start_time) / 100
print(f"Average inference time per image: {avg_inference_time:.4f} seconds")


FLOPs: 1818554880
Average inference time per image: 0.0030 seconds


##MobileNet V2

In [20]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    train_loss_history = []
    val_acc_history = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        train_loss_history.append(avg_train_loss)

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        val_acc_history.append(val_accuracy)

        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

    print("Training complete.")
    return model, (train_loss_history, val_acc_history)


In [21]:
import torchvision.models as models
import torch.nn as nn

# Load pretrained MobileNetV2 model
mobilenet = models.mobilenet_v2(pretrained=True)

# Freeze all layers (optional)
for param in mobilenet.features.parameters():
    param.requires_grad = False

# Modify the classifier to match the number of output classes (2: fire, nofire)
mobilenet.classifier[1] = nn.Linear(mobilenet.last_channel, 2)

# Move to device
mobilenet = mobilenet.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mobilenet.parameters(), lr=0.001)

# Train the model
mobilenet, history = train_model(mobilenet, train_loader, val_loader, criterion, optimizer, 10)


Epoch [1/10], Train Loss: 0.2407, Val Acc: 96.25%
Epoch [2/10], Train Loss: 0.1037, Val Acc: 98.75%
Epoch [3/10], Train Loss: 0.0939, Val Acc: 98.75%
Epoch [4/10], Train Loss: 0.0659, Val Acc: 98.75%
Epoch [5/10], Train Loss: 0.0514, Val Acc: 98.75%
Epoch [6/10], Train Loss: 0.0451, Val Acc: 98.75%
Epoch [7/10], Train Loss: 0.0397, Val Acc: 97.50%
Epoch [8/10], Train Loss: 0.0468, Val Acc: 98.75%
Epoch [9/10], Train Loss: 0.0429, Val Acc: 98.75%
Epoch [10/10], Train Loss: 0.0380, Val Acc: 97.50%
Training complete.


In [22]:
from torchsummary import summary
summary(mobilenet, input_size=(3, 224, 224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

In [23]:
from fvcore.nn import FlopCountAnalysis

dummy_input = torch.randn(1, 3, 224, 224).to(device)
flops = FlopCountAnalysis(mobilenet, dummy_input)
print(f"FLOPs: {flops.total()}")  # Or use flops.pretty_print() for formatted


FLOPs: 312915776


In [24]:
import time

mobilenet.eval()
start_time = time.time()

with torch.no_grad():
    for _ in range(100):
        _ = mobilenet(dummy_input)

end_time = time.time()
avg_inference_time = (end_time - start_time) / 100
print(f"Average inference time per image: {avg_inference_time:.4f} seconds")


Average inference time per image: 0.0056 seconds


## EfficentNetB0

In [25]:
from torchvision import models
import torch.optim as optim

# Load the EfficientNetB0 model
efficientnet = models.efficientnet_b0(pretrained=True)
efficientnet.classifier[1] = torch.nn.Linear(efficientnet.classifier[1].in_features, 2)  # Adjust for 2 output classes

# Send the model to the device
efficientnet = efficientnet.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(efficientnet.parameters(), lr=0.001)

# Train the model
efficientnet, history = train_model(efficientnet, train_loader, val_loader, criterion, optimizer, 10)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 144MB/s]


Epoch [1/10], Train Loss: 0.1035, Val Acc: 100.00%
Epoch [2/10], Train Loss: 0.0402, Val Acc: 100.00%
Epoch [3/10], Train Loss: 0.0296, Val Acc: 100.00%
Epoch [4/10], Train Loss: 0.0154, Val Acc: 98.75%
Epoch [5/10], Train Loss: 0.0224, Val Acc: 100.00%
Epoch [6/10], Train Loss: 0.0255, Val Acc: 100.00%
Epoch [7/10], Train Loss: 0.0140, Val Acc: 100.00%
Epoch [8/10], Train Loss: 0.0344, Val Acc: 98.75%
Epoch [9/10], Train Loss: 0.0168, Val Acc: 100.00%
Epoch [10/10], Train Loss: 0.0231, Val Acc: 100.00%
Training complete.


In [27]:
summary(efficientnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              SiLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              SiLU-6         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 112, 112]               0
           Conv2d-13         [-1, 16, 112, 112]             512
      BatchNorm2d-14         [-1, 16, 1

In [28]:
import torch
import time
from fvcore.nn import FlopCountAnalysis

# Move model to device
efficientnet = efficientnet.to(device)

# Create a dummy input
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# Calculate FLOPs
flops = FlopCountAnalysis(efficientnet, dummy_input)
print(f"FLOPs: {flops.total()}")

# Measure inference time
start_time = time.time()
with torch.no_grad():
    efficientnet(dummy_input)
end_time = time.time()

inference_time = end_time - start_time
print(f"Average inference time per image: {inference_time:.4f} seconds")


features.1.0.stochastic_depth, features.2.0.stochastic_depth, features.2.1.stochastic_depth, features.3.0.stochastic_depth, features.3.1.stochastic_depth, features.4.0.stochastic_depth, features.4.1.stochastic_depth, features.4.2.stochastic_depth, features.5.0.stochastic_depth, features.5.1.stochastic_depth, features.5.2.stochastic_depth, features.6.0.stochastic_depth, features.6.1.stochastic_depth, features.6.2.stochastic_depth, features.6.3.stochastic_depth, features.7.0.stochastic_depth


FLOPs: 400381952
Average inference time per image: 0.0135 seconds
